In [24]:
import wget
fitxers = [
    "https://icaen.gencat.cat/web/.content/20_Energia/26_ajuts_financament/2021_IDAE_EERR/Arxius/Resolucio_Atorgament_PUBLICABLE-st_ah_v3_L1_1.pdf",
]

url = fitxers[0]
wget.download(url, './data/f1.pdf')


'./data/f1.pdf'

In [25]:
import PyPDF2

# creating an object 
file = open('./data/f1.pdf', 'rb')

# creating a pdf reader object
reader = PyPDF2.PdfFileReader(file)

s = ""
for page in list(reader.pages):
    s += page.extract_text()

from_char = s.find("Codi expedient ")
data = s[from_char:].replace("\n", "").split(" ICA")[1:]

In [26]:
def tokenize(item):
    """025/21/000332  ***0530**  GESTINGRAL SL  Instal.lació de generació de 9,57 kW  Vilanova i la Geltrú  11369,16  5.116,12  0,00  15/12/2021 09:25:44 
    098765432109876543210987654321098765432109876543210987654321098765432109876543210987654321098765432109876543210987654321098765432109876543210987654321
    """

    if item[15:18] != "***":
        return None

    last_second = item.rfind(":")
    item1 = item[:last_second + 3]
    dia = item1[-19:]
    item2 = item1[:-22].replace("  ", " ")

    items = item2.split(" ")
    inversio_justificar = float( items[-3].replace(".", "").replace(",", ".") )

    items = items[:-3]
    item3 = " ".join(items)

    from_kWh = item3.rfind("kW") + 3
    poblacio = item3[from_kWh:]

    return {
        "dia": dia,
        "inversio_justificar": inversio_justificar,
        "poblacio": poblacio,
        "linia": item
        }


recordset = [tokenize(d) for d in data]



In [27]:
import pandas as pd

df = pd.DataFrame(recordset)


In [28]:
import altair as alt

(
    alt
    .Chart(df)
    .mark_bar()
    .encode(
        alt.X("sum(inversio_justificar):Q",title="Inversió a justificar", stack='zero'),
        alt.Y("poblacio:N", sort="-x", title=""),
    )
)


alt.Chart(...)